In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gunavenkatdoddi/eye-diseases-classification")

print("Path to dataset files:", path)

100%|██████████| 736M/736M [00:06<00:00, 115MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1


In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir="/root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1/dataset")

Resolving data files:   0%|          | 0/4217 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'glaucoma'

In [6]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [7]:
def preprocess_images(example):
    inputs = processor(example['image'], return_tensors="pt")
    example['pixel_values'] = inputs['pixel_values'].squeeze()  # Remove extra batch dimension
    return example

In [8]:
processed_dataset = dataset.map(preprocess_images)

Map:   0%|          | 0/4217 [00:00<?, ? examples/s]

In [9]:
splits = processed_dataset["train"].train_test_split(test_size=0.3)
val_test_split = splits['test'].train_test_split(test_size=0.5)  # 15% validation, 15% test

train_ds = splits['train']
val_ds = val_test_split['train']
test_ds = val_test_split['test']

In [10]:
train_ds

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 2951
})

In [11]:
val_ds

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 633
})

In [12]:
test_ds

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 633
})

In [13]:
label2id

{'cataract': 0, 'diabetic_retinopathy': 1, 'glaucoma': 2, 'normal': 3}

In [14]:
id2label

{0: 'cataract', 1: 'diabetic_retinopathy', 2: 'glaucoma', 3: 'normal'}

In [15]:
model = AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Mediscan'/Vit",
    per_device_train_batch_size=16,
    eval_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=200,
    eval_steps=200,
    logging_steps=100,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)


In [35]:
!pip install evaluate

In [38]:
import torch
import numpy as np

from evaluate import load
# Collate function for creating batches
def collate_fn(examples):
    pixel_values = torch.stack([torch.tensor(example["pixel_values"]) for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# Define the accuracy metric
metric = load("accuracy")
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [40]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
200,0.296300,0.277794,0.897314
400,0.150400,0.256684,0.927330
600,0.063000,0.317729,0.933649


***** train metrics *****
  epoch                    =         4.0
  total_flos               = 851910414GF
  train_loss               =      0.1619
  train_runtime            =  0:27:08.43
  train_samples_per_second =       7.249
  train_steps_per_second   =       0.454


In [46]:
print("Train Results:", train_results)


Train Results: TrainOutput(global_step=740, training_loss=0.16191181161919155, metrics={'train_runtime': 1628.4367, 'train_samples_per_second': 7.249, 'train_steps_per_second': 0.454, 'total_flos': 9.147318420210647e+17, 'train_loss': 0.16191181161919155, 'epoch': 4.0})


In [47]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)
print("Evaluate Results:", metrics)


Exception ignored in: <function _xla_gc_callback at 0x78b90e87bd90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9273
  eval_loss               =     0.2567
  eval_runtime            = 0:01:08.36
  eval_samples_per_second =      9.259
  eval_steps_per_second   =       1.17
Evaluate Results: {'eval_loss': 0.2566837668418884, 'eval_accuracy': 0.9273301737756714, 'eval_runtime': 68.3636, 'eval_samples_per_second': 9.259, 'eval_steps_per_second': 1.17, 'epoch': 4.0}


In [48]:
test_results = trainer.evaluate(test_ds)
print("Test Results:", test_results)

Test Results: {'eval_loss': 0.30371934175491333, 'eval_accuracy': 0.9115323854660348, 'eval_runtime': 71.1173, 'eval_samples_per_second': 8.901, 'eval_steps_per_second': 1.125, 'epoch': 4.0}
